# SCC-ICMC-USP - 1o. semestre de 2024
# SCC5871/MAI5025 - APRENDIZADO DE MÁQUINA
# Exercício 3

### Profa. Roseli A. F. Romero


Nro do grupo:

Alunos:


1.   Julyana Flores de Prá
2.   Thiago Rafael Mariotti Claudio

## Objetivo: Curva ROC e Teste de hipótese;
##           Utilização de Perceptron, MLP, DT e KNN

## Funções novas utilizadas no exercício

- `pandas.Series.nunique()` ([link](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.nunique.html)): Conta quantidade de valores únicos de uma coluna. Útil para verificar se uma coluna é relevante ou não
- `scipy.stats.ttest_ind()` ([link](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html#scipy.stats.ttest_ind)): Calcula o teste t para duas amostras independentes
- `sklearn.metrics.plot_roc_curve()` ([link](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_roc_curve.html)): Plota a curva ROC de um classificador dado um conjunto de input e alvo

### Questão 01.

Faça a exploração dos dados. Isto é, carregue, substitua valores faltantes, padronize os dados, etc. Faça também a seleção dos atributos que achar mais relevantes.

 - Dica: Utilize a função `nunique()` durante a exploração dos dados. Você pode utilizar o "bom senso" (além de outras ferramentas é claro) na hora de escolher qual atributo do conjunto manter

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

df = pd.read_csv("train.csv")

df["Survived"] = pd.to_numeric(df["Survived"], errors="coerce")

#male = 1, female = 0
df['Sex_Encoded'] = label_encoder.fit_transform(df['Sex'])
df

In [ ]:
# Etapas de pré-processamento que podem ser feitas antes de separar em treino/teste vão aqui
# código de solução
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.nunique()

In [ ]:
df.isnull().sum()

In [ ]:
mean_age = df['Age'].mean()
df['Age'].fillna(mean_age, inplace=True)

In [ ]:
attributes = ["Pclass", "Fare", "Survived", "Age", "Sex_Encoded", "SibSp", "Parch"]
correlation = df[attributes].corr()
correlation

In [ ]:
import seaborn as sns
%matplotlib inline
sns.set_style("whitegrid")

sns.heatmap(correlation, annot=True)

### Questão 02.

- a) Separe o conjunto de dados de maneira estratificada (através do parâmetro `stratify` da função `train_test_split`) em 20% para teste e 80% para treino. Depois plote a curva ROC (`sklearn.metrics.plot_roc_curve`) para **todos** os classificadores (no mesmo gráfico).

- b) Os melhores classificadores da questão anterior também apresentaram melhor desempenho na curva ROC? O que pode ter ocorrido? Teste diferentes valores de `random_state` na função `train_test_split` e observe o comportamento das curvas.


*   Dica: Para plotar múltiplas curvas ROC no mesmo gráfico, defina uma figura com `fig, ax = plt.subplots()` e passe `ax` como parâmetro da função `plot_roc_curve`. Não se esqueça de passar também o nome do classificador para que o seu gráfico fique mais fácil de interpretar



In [ ]:
from sklearn.preprocessing import StandardScaler

# Separar conjunto e pré processamento
df_novo = df[attributes].copy(deep=True)
X = df_novo.drop(columns=['Survived'])  # Supondo que 'Survived' seja o atributo alvo
y = df_novo['Survived']

# Pré-processamento: padronizar os dados
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np

classificadores = {
  "Perceptron" : {"modelo": Perceptron(), "scores": []},
  "Multi-Layer Perceptron (15,)" : {"modelo": MLPClassifier(random_state=1, hidden_layer_sizes=(15,), max_iter=2000), "scores": []},
  "Árvore Decisão Critério Gini" : {"modelo": DecisionTreeClassifier(criterion='gini'), "scores": []},
  "KNN k=5" : {"modelo": KNeighborsClassifier(n_neighbors=5), "scores": []}
}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

plt.rcParams['figure.figsize'] = [15, 10]
fig, ax = plt.subplots()

for classificador_name, classificador_info in classificadores.items():
    clf = classificador_info["modelo"]
    
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    
    roc_auc = metrics.roc_auc_score(y_test, y_pred)
    classificador_info["scores"].append(roc_auc)
    
    metrics.plot_roc_curve(clf, X_test, y_test, ax=ax, name=classificador_name)

plt.title('Curva ROC para todos os classificadores')
plt.xlabel('Taxa de Falso Positivo')
plt.ylabel('Taxa de Verdadeiro Positivo')
plt.legend(loc='lower right')
plt.show()

### Questão 03.

Implemente o 10-Fold Cross Validation (pode usar o scikit) com os dois melhores classificadores de acordo com a curva ROC e guarde a acurácia de cada fold na chave 'scores' do dicionário de classificadores.

In [ ]:
# Seu código aqui
# Lembre-se do pré-processamento

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

melhores_classificadores = sorted(classificadores.items(), key=lambda x: np.mean(x[1]["scores"]), reverse=True)[:2]

kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for classificador_name, classificador_info in melhores_classificadores:
    clf = classificador_info["modelo"]
    
    scores = cross_val_score(clf, X, y, cv=kf)
    
    classificador_info["scores"] = scores

# Imprimir as acurácias de cada fold para os melhores classificadores
for classificador_name, classificador_info in melhores_classificadores:
    print(f"Classificador: {classificador_name}")
    print(f"Acurácias dos folds: {classificador_info['scores']}")
    print(f"Acurácia média: {np.mean(classificador_info['scores'])}")
    print()

### Questão 04.
Altere a configuração da rede MLP, da arvore de Decisão (use outra função de entropia, o no. de vizinhos mais proximo no método KNN. Verifique o desempenho de cada um deles e compare com cada caso anterior.

In [ ]:
classificadores_modificados = {
    "Perceptron": {"modelo": Perceptron(), "scores": []},
    "MLP (20, 10)": {"modelo": MLPClassifier(random_state=1, hidden_layer_sizes=(20, 10), max_iter=2000), "scores": []},
    "Árvore Decisão (entropy)": {"modelo": DecisionTreeClassifier(criterion='entropy'), "scores": []},
    "KNN k=10": {"modelo": KNeighborsClassifier(n_neighbors=10), "scores": []}
}

for classificador_name, classificador_info in classificadores_modificados.items():
    clf = classificador_info["modelo"]
    
    scores = cross_val_score(clf, X, y, cv=kf)
    
    classificador_info["scores"] = scores

for classificador_name, classificador_info in classificadores_modificados.items():
    print(f"Classificador: {classificador_name}")
    print(f"Acurácias dos folds: {classificador_info['scores']}")
    print(f"Acurácia média: {np.mean(classificador_info['scores'])}")
    print()

### Questão 05.

Verifique se há diferença estatística significante entre suas acurácias da questão anterior utilizando o teste T (`scipy.stats.ttest_ind`). Considere que há diferença significante se p <= 0.05 (rejeita-se a hipótese nula)

In [ ]:
from scipy.stats import ttest_ind

acuracias_por_classificador = []

for classificador_info in classificadores_modificados.values():
    acuracias_por_classificador.append(classificador_info["scores"])

p_value, _ = ttest_ind(acuracias_por_classificador[0], acuracias_por_classificador[1])

# Verificar se o p-valor é menor ou igual a 0.05
if p_value <= 0.05:
    print("Há diferença estatisticamente significativa entre as acurácias.")
else:
    print("Não há diferença estatisticamente significativa entre as acurácias.")